In [45]:
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV
import math
import pandas as pd
import numpy as np

df = pd.read_csv('dataset3.csv')

print(df.head(), df.shape)
# separation des features et de la variable cible
X = df.drop('charges', axis=1)
y = df[['charges']]
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')


   age  sex     bmi  children  smoker     region      charges  \
0   19    1  27.900         0       1  southwest  16884.92400   
1   18    0  33.770         1       0  southeast   1725.55230   
2   28    0  33.000         3       0  southeast   4449.46200   
3   33    0  22.705         0       0  northwest  21984.47061   
4   32    0  28.880         0       0  northwest   3866.85520   

   bmi_smoker_interaction  
0                    27.9  
1                     0.0  
2                     0.0  
3                     0.0  
4                     0.0   (1337, 8)
verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 7)
      Y (la variable cible) : (1337, 1)
 verif du split 80 20 
80% du dataset : X train -> (1136, 7), Y train -> (1136, 1)
20% du dataset : X test -> (201, 7), Y test -> (201, 1)


In [46]:
#Elasticnet
# préprocesseur pour les variables numériques
def log_transform(x):
    return np.log(x + 1)

log_transformer = FunctionTransformer(log_transform)

preprocessor_num = Pipeline(steps=[
    ('log', log_transformer),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])
preprocessor_cat = Pipeline(steps=[
    ('encoder', OneHotEncoder()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
])

# preprocessing avec transformation des categories et des nums
preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessor_num , ['age', 'bmi', 'children', 'bmi_smoker_interaction']),
        ('cat', preprocessor_cat , ['region','sex', 'smoker']),
    ]
)
# création du pipeline de prétraitement et du modèle elasticnet
pipeline_elasticnet = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regression', ElasticNet())])


#test de différents hyperparam d'alpha pour le lasso
param_grid_elasticnet = {
    'regression__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'regression__l1_ratio' : [0.1, 0.3, 0.5, 0.7, 0.9, 1]
}

### grid search
grid_elasticnet = GridSearchCV(pipeline_elasticnet, param_grid_elasticnet, cv=10)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_elasticnet.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_elasticnet.predict(X_test)
# print(y_pred)


/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.300e+10, tolerance: 1.518e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.308e+10, tolerance: 1.465e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

In [47]:
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)


r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle elasticnet: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle elasticnet: {rmse} \n rappel moyenne charge : 13279')

Coefficient de determination R² du modèle elasticnet: 0.9036438286997107
Root Mean Squared Error (RMSE) du modèle elasticnet: 3736.7807743347416 
 rappel moyenne charge : 13279


In [48]:
best_model = grid_elasticnet.best_estimator_
best_model


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('log',
                                                                   FunctionTransformer(func=<function log_transform at 0x7f44f06c71c0>)),
                                                                  ('poly',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bmi', 'children',
                                                   'bmi_smoker_interaction']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder()),
                                                                  ('poly',
                                                                   PolynomialFeatures(include_bias=False))]),
                                                  ['region', 'sex',
                                                   'smoker'])])),
                ('regression', ElasticNet(alpha=1, l1_ratio=1))])

In [49]:
new_data = pd.DataFrame({
    'age': [35, 40, 30, 60,60],  
    'sex': [1, 0, 0, 1,1],  
    'bmi': [22.5, 30.0, 25.0, 47,50],
    'children': [1, 2, 0, 5,5],
    'smoker': [1, 0, 0, 1,1],  
    'region': ['southwest', 'northeast', 'southeast', 'southeast', 'southeast'],
    'bmi_smoker_interaction': [22.5,0,0,47,50]
})

# faire des prédictions avec le modèle optimisé, possible de faire best_model.predict aussi 
new_y_ped = grid_elasticnet.predict(new_data)

# ajouter les prédictions à new_data
new_data['predicted_charges'] = new_y_ped

print(new_data)

   age  sex   bmi  children  smoker     region  bmi_smoker_interaction  \
0   35    1  22.5         1       1  southwest                    22.5   
1   40    0  30.0         2       0  northeast                     0.0   
2   30    0  25.0         0       0  southeast                     0.0   
3   60    1  47.0         5       1  southeast                    47.0   
4   60    1  50.0         5       1  southeast                    50.0   

   predicted_charges  
0       18410.724741  
1        9468.707773  
2        4152.824602  
3       59293.102551  
4       62276.613013  


In [50]:
ElasticNet?

Init signature:
ElasticNet(
    alpha=1.0,
    *,
    l1_ratio=0.5,
    fit_intercept=True,
    precompute=False,
    max_iter=1000,
    copy_X=True,
    tol=0.0001,
    warm_start=False,
    positive=False,
    random_state=None,
    selection='cyclic',
)
Docstring:     
Linear regression with combined L1 and L2 priors as regularizer.

Minimizes the objective function::

        1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

If you are interested in controlling the L1 and L2 penalty
separately, keep in mind that this is equivalent to::

        a * ||w||_1 + 0.5 * b * ||w||_2^2

where::

        alpha = a + b and l1_ratio = a / (a + b)

The parameter l1_ratio corresponds to alpha in the glmnet R package while
alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio
= 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable,
unless you supply your own sequence of alpha.

Read mo

In [51]:
import pickle
with open('modele_elasticnet.pkl', 'wb') as fichier:
    pickle.dump(grid_elasticnet, fichier)